In [1]:
import matplotlib
# matplotlib.use('Agg')
import matplotlib.pyplot as plt
import copy
import numpy as np
from torchvision import datasets, transforms
import torch

from utils.sampling import mnist_iid, mnist_noniid, cifar_iid, cifar_noniid, cifar_noniid_shared
from utils.options import args_parser
from models.Update import LocalUpdate
from models.Nets import MLP, CNNMnist, CNNCifar, LeNet, CNNMnist2
from models.Fed import FedAvg
from models.Fed import FedQAvg, Quantization, Quantization_Finite, my_score, my_score_Finite
from models.test import test_img


import math


# from sympy import * 
from utils.functions import *

%load_ext autoreload
%autoreload 2

In [2]:
class my_argument:    
    epochs = 200    #"rounds of training"
    num_users = 40  # "number of users: K"
    frac = 0.5 #"the fraction of clients: C"
    local_ep=1 #"the number of local epochs: E"
    local_bs=100 #"local batch size: B"
    bs=100 #"test batch size"
    lr=0.03 #"learning rate"
    momentum=0.5 # "SGD momentum (default: 0.5)"
    split='user' # "train-test split type, user or sample"
    weight_decay = 5e-4
    opt = 'SGD' #'ADAM'
    loss = 'Cross'

    # model arguments
    model = 'cnn'
    kernel_num=9 #, help='number of each kind of kernel')
    kernel_sizes='3,4,5' #  help='comma-separated kernel size to use for convolution')
    norm='batch_norm' #, help="batch_norm, layer_norm, or None")
    num_filters=32 #, help="number of filters for conv nets")
    max_pool='True' #help="Whether use max pooling rather than strided convolutions")

    # other arguments
    dataset='cifar' #, help="name of dataset")
    iid=0
    num_classes=10#, help="number of classes")
    num_channels=1#, help="number of channels of imges")
    gpu=1#, help="GPU ID, -1 for CPU")
    stopping_rounds=10#, help='rounds of early stopping')
    verbose='False'#, help='verbose print')
    seed=1#, help='random seed (default: 1)')
    
args = my_argument()

args.device = torch.device('cuda:{}'.format(args.gpu) if torch.cuda.is_available() and args.gpu != -1 else 'cpu')

use_cuda = torch.cuda.is_available()
print(use_cuda)
args.device = torch.device("cuda" if use_cuda else "cpu")
print(args.device)

True
cuda


In [3]:
from torchvision import datasets, transforms


trans_cifar = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
dataset_train = datasets.CIFAR10('./data/cifar', train=True, download=True, transform=trans_cifar)
dataset_test = datasets.CIFAR10('./data/cifar', train=False, download=True, transform=trans_cifar)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
dict_users = cifar_noniid_shared(dataset_train, args.num_users)

number of non-shared images= 1245
number of shared images= 200
number of images per user= 1445


In [5]:
N = 40
K = 8

# p_per_class = np.array([-0.1, -0.05, 0 , 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35])+0.025

p_per_class = np.array([0.1, 0.15, 0.2, 0.25, 0.3, 0.3, 0.35, 0.4, 0.45, 0.5])
print(p_per_class)

p_per_user = np.zeros((N,))

hist_ = np.zeros((10,))

for i in range(N):
    idxs=dict_users[i].astype(int)
    target = np.array(dataset_train.targets)
    cur_labels = target[idxs]
#     print(np.shape(target))
#     print(target[idxs])
    
    label_first = int(cur_labels[0])
    label_second = int(cur_labels[-1])
    
#     print(i,cur_labels)
    print(i,label_first,label_second)
    
    hist_[label_first] += 1
    
    p_per_user[i] = p_per_class[label_first]
#     print(p_per_class[label_first] + p_per_class[label_second], p_per_user[i])
print(p_per_user)

print("hist",hist_)

print(np.sum(p_per_user)/120)

[0.1  0.15 0.2  0.25 0.3  0.3  0.35 0.4  0.45 0.5 ]
0 9 9
1 1 9
2 7 9
3 6 9
4 9 9
5 3 9
6 7 9
7 0 9
8 5 9
9 7 9
10 1 9
11 6 9
12 4 9
13 9 9
14 5 9
15 5 9
16 1 9
17 9 9
18 6 9
19 2 9
20 0 9
21 3 9
22 8 9
23 1 9
24 3 9
25 8 9
26 4 9
27 8 9
28 0 9
29 3 9
30 8 9
31 4 9
32 2 9
33 0 9
34 6 9
35 2 9
36 5 9
37 4 9
38 7 9
39 2 9
[0.5  0.15 0.4  0.35 0.5  0.25 0.4  0.1  0.3  0.4  0.15 0.35 0.3  0.5
 0.3  0.3  0.15 0.5  0.35 0.2  0.1  0.25 0.45 0.15 0.25 0.45 0.3  0.45
 0.1  0.25 0.45 0.3  0.2  0.1  0.35 0.2  0.3  0.3  0.4  0.2 ]
hist [4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]
0.1


In [10]:
from models.Nets import NIN,CNN_moderate, CNNCifar3

p = 0.3
N = 40
K = 8

N_trials = 1
Max_iter = 1200

args.opt = 'SGD'

lr_array = [0.01]

args.local_ep = 1
args.local_bs = 50
args.weight_decay = 5e-4

acc_test_arr_random  = np.zeros((len(lr_array),N_trials,Max_iter))
loss_test_arr_random = np.zeros((len(lr_array),N_trials,Max_iter))

P_random = []

for lr_idx in range(len(lr_array)):
    
    for trial_idx in range(N_trials):
        
        args.lr       = lr_array[lr_idx]
        
        net_glob = CNNCifar(args)
        
        net_glob = net_glob.cuda()
        
        print(net_glob)

        net_glob.train()
        
        P_random = []

        # copy weights
        w_glob = net_glob.state_dict()
        for iter in range(Max_iter): #args.epochs
            
            
            if iter == 400 or iter == 800:
                args.lr = args.lr * 0.4
            
            print('lr=',args.lr)
            
            w_locals, loss_locals = [], []

            ###############################
            # 0. Dropout Realization
            ###############################    
            
            u = np.ones((N,))
            for u_idx in range(N):
                p_sel = p_per_user[u_idx]
                u[u_idx] = np.random.binomial(1, 1-p_sel, size=1)[0]
            
            result = np.where(u == 1)
            drop_result = np.where(u == 0)

            ###############################
            # 1. Weighted Random Selection
            ###############################

            #idxs_users = np.random.choice(range(N), K, replace=False)
            
            idxs_users = np.random.choice(result[0], K, replace=False)


            p_tmp = np.zeros(N)
            p_tmp[idxs_users] = 1

            P_random.append(p_tmp)

        #     idxs_users = np.random.choice(range(N), K, replace=False)
            for idx in idxs_users:
        #         print(idx)
                local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[idx])
                w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))
                w_locals.append(copy.deepcopy(w))
                loss_locals.append(copy.deepcopy(loss))
            # update global weights
            w_glob = FedAvg(w_locals)

            # copy weight to net_glob
            net_glob.load_state_dict(w_glob)

            # print loss
            loss_avg = sum(loss_locals) / len(loss_locals)

            #loss_train.append(loss_avg)

            acc_test, loss_test = test_img(net_glob, dataset_test, args)
            acc_test_arr_random[lr_idx][trial_idx][iter]  = acc_test
            loss_test_arr_random[lr_idx][trial_idx][iter] = loss_test
            if iter % 1 ==0:
                print('Round {:3d}, Train average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_avg,acc_test))
            #print(loss_train)
            
            if iter % 100 == 99:
                PATH = "./save_models/CIFAR10_NonIID_CNN_N40_K8_net_glob_iter"+str(iter)
                torch.save(net_glob.state_dict(), PATH)

CNNCifar(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
lr= 0.01

Test set: Average loss: 2.3062 
Accuracy: 1000/10000 (10.00%)

Round   0, Train average loss 2.079 Test accuracy 10.000
lr= 0.01

Test set: Average loss: 2.2969 
Accuracy: 1239/10000 (12.39%)

Round   1, Train average loss 1.746 Test accuracy 12.390
lr= 0.01

Test set: Average loss: 2.3186 
Accuracy: 1000/10000 (10.00%)

Round   2, Train average loss 1.219 Test accuracy 10.000
lr= 0.01

Test set: Average loss: 2.3022 
Accuracy: 1360/10000 (13.60%)

Round   3, Train average loss 1.088 Test accuracy 13.600
lr= 0.01

Test set: Average loss: 2.2962 
Accuracy: 1329/10000 (13.29%)

Round   


Test set: Average loss: 2.0578 
Accuracy: 2074/10000 (20.74%)

Round  60, Train average loss 0.713 Test accuracy 20.740
lr= 0.01

Test set: Average loss: 2.0570 
Accuracy: 2039/10000 (20.39%)

Round  61, Train average loss 0.732 Test accuracy 20.390
lr= 0.01

Test set: Average loss: 2.0445 
Accuracy: 2431/10000 (24.31%)

Round  62, Train average loss 0.704 Test accuracy 24.310
lr= 0.01

Test set: Average loss: 2.0510 
Accuracy: 2094/10000 (20.94%)

Round  63, Train average loss 0.702 Test accuracy 20.940
lr= 0.01

Test set: Average loss: 2.0407 
Accuracy: 2452/10000 (24.52%)

Round  64, Train average loss 0.726 Test accuracy 24.520
lr= 0.01

Test set: Average loss: 2.0442 
Accuracy: 2486/10000 (24.86%)

Round  65, Train average loss 0.685 Test accuracy 24.860
lr= 0.01

Test set: Average loss: 2.0357 
Accuracy: 2430/10000 (24.30%)

Round  66, Train average loss 0.703 Test accuracy 24.300
lr= 0.01

Test set: Average loss: 2.0315 
Accuracy: 2281/10000 (22.81%)

Round  67, Train average l


Test set: Average loss: 1.9547 
Accuracy: 2754/10000 (27.54%)

Round 124, Train average loss 0.621 Test accuracy 27.540
lr= 0.01

Test set: Average loss: 1.9507 
Accuracy: 2804/10000 (28.04%)

Round 125, Train average loss 0.639 Test accuracy 28.040
lr= 0.01

Test set: Average loss: 1.9530 
Accuracy: 2747/10000 (27.47%)

Round 126, Train average loss 0.648 Test accuracy 27.470
lr= 0.01

Test set: Average loss: 1.9485 
Accuracy: 2685/10000 (26.85%)

Round 127, Train average loss 0.636 Test accuracy 26.850
lr= 0.01

Test set: Average loss: 1.9551 
Accuracy: 2656/10000 (26.56%)

Round 128, Train average loss 0.630 Test accuracy 26.560
lr= 0.01

Test set: Average loss: 1.9488 
Accuracy: 2811/10000 (28.11%)

Round 129, Train average loss 0.621 Test accuracy 28.110
lr= 0.01

Test set: Average loss: 1.9346 
Accuracy: 2915/10000 (29.15%)

Round 130, Train average loss 0.630 Test accuracy 29.150
lr= 0.01

Test set: Average loss: 1.9297 
Accuracy: 2927/10000 (29.27%)

Round 131, Train average l


Test set: Average loss: 1.8501 
Accuracy: 3109/10000 (31.09%)

Round 188, Train average loss 0.463 Test accuracy 31.090
lr= 0.01

Test set: Average loss: 1.8600 
Accuracy: 3115/10000 (31.15%)

Round 189, Train average loss 0.559 Test accuracy 31.150
lr= 0.01

Test set: Average loss: 1.8449 
Accuracy: 3287/10000 (32.87%)

Round 190, Train average loss 0.523 Test accuracy 32.870
lr= 0.01

Test set: Average loss: 1.8545 
Accuracy: 3236/10000 (32.36%)

Round 191, Train average loss 0.535 Test accuracy 32.360
lr= 0.01

Test set: Average loss: 1.8505 
Accuracy: 3273/10000 (32.73%)

Round 192, Train average loss 0.494 Test accuracy 32.730
lr= 0.01

Test set: Average loss: 1.8389 
Accuracy: 3255/10000 (32.55%)

Round 193, Train average loss 0.510 Test accuracy 32.550
lr= 0.01

Test set: Average loss: 1.8394 
Accuracy: 3169/10000 (31.69%)

Round 194, Train average loss 0.490 Test accuracy 31.690
lr= 0.01

Test set: Average loss: 1.8476 
Accuracy: 3190/10000 (31.90%)

Round 195, Train average l


Test set: Average loss: 1.8298 
Accuracy: 3516/10000 (35.16%)

Round 252, Train average loss 0.432 Test accuracy 35.160
lr= 0.01

Test set: Average loss: 1.8249 
Accuracy: 3478/10000 (34.78%)

Round 253, Train average loss 0.409 Test accuracy 34.780
lr= 0.01

Test set: Average loss: 1.8217 
Accuracy: 3491/10000 (34.91%)

Round 254, Train average loss 0.406 Test accuracy 34.910
lr= 0.01

Test set: Average loss: 1.8309 
Accuracy: 3423/10000 (34.23%)

Round 255, Train average loss 0.410 Test accuracy 34.230
lr= 0.01

Test set: Average loss: 1.8240 
Accuracy: 3504/10000 (35.04%)

Round 256, Train average loss 0.365 Test accuracy 35.040
lr= 0.01

Test set: Average loss: 1.8191 
Accuracy: 3524/10000 (35.24%)

Round 257, Train average loss 0.419 Test accuracy 35.240
lr= 0.01

Test set: Average loss: 1.8140 
Accuracy: 3457/10000 (34.57%)

Round 258, Train average loss 0.367 Test accuracy 34.570
lr= 0.01

Test set: Average loss: 1.8153 
Accuracy: 3532/10000 (35.32%)

Round 259, Train average l

...


Test set: Average loss: 2.2979 
Accuracy: 3801/10000 (38.01%)

Round 947, Train average loss 0.099 Test accuracy 38.010
lr= 0.0016

Test set: Average loss: 2.3102 
Accuracy: 3808/10000 (38.08%)

Round 948, Train average loss 0.099 Test accuracy 38.080
lr= 0.0016

Test set: Average loss: 2.2849 
Accuracy: 3784/10000 (37.84%)

Round 949, Train average loss 0.101 Test accuracy 37.840
lr= 0.0016

Test set: Average loss: 2.2979 
Accuracy: 3896/10000 (38.96%)

Round 950, Train average loss 0.079 Test accuracy 38.960
lr= 0.0016

Test set: Average loss: 2.4062 
Accuracy: 3713/10000 (37.13%)

Round 951, Train average loss 0.112 Test accuracy 37.130
lr= 0.0016

Test set: Average loss: 2.3116 
Accuracy: 3819/10000 (38.19%)

Round 952, Train average loss 0.102 Test accuracy 38.190
lr= 0.0016

Test set: Average loss: 2.3326 
Accuracy: 3768/10000 (37.68%)

Round 953, Train average loss 0.091 Test accuracy 37.680
lr= 0.0016

Test set: Average loss: 2.4123 
Accuracy: 3859/10000 (38.59%)

Round 954, T


Test set: Average loss: 2.4356 
Accuracy: 3755/10000 (37.55%)

Round 1009, Train average loss 0.089 Test accuracy 37.550
lr= 0.0016

Test set: Average loss: 2.3595 
Accuracy: 3784/10000 (37.84%)

Round 1010, Train average loss 0.090 Test accuracy 37.840
lr= 0.0016

Test set: Average loss: 2.3541 
Accuracy: 3816/10000 (38.16%)

Round 1011, Train average loss 0.083 Test accuracy 38.160
lr= 0.0016

Test set: Average loss: 2.4773 
Accuracy: 3662/10000 (36.62%)

Round 1012, Train average loss 0.089 Test accuracy 36.620
lr= 0.0016

Test set: Average loss: 2.4909 
Accuracy: 3687/10000 (36.87%)

Round 1013, Train average loss 0.072 Test accuracy 36.870
lr= 0.0016

Test set: Average loss: 2.6089 
Accuracy: 3530/10000 (35.30%)

Round 1014, Train average loss 0.090 Test accuracy 35.300
lr= 0.0016

Test set: Average loss: 2.3953 
Accuracy: 3751/10000 (37.51%)

Round 1015, Train average loss 0.086 Test accuracy 37.510
lr= 0.0016

Test set: Average loss: 2.5031 
Accuracy: 3699/10000 (36.99%)

Round


Test set: Average loss: 2.3750 
Accuracy: 3773/10000 (37.73%)

Round 1071, Train average loss 0.103 Test accuracy 37.730
lr= 0.0016

Test set: Average loss: 2.3682 
Accuracy: 3749/10000 (37.49%)

Round 1072, Train average loss 0.095 Test accuracy 37.490
lr= 0.0016

Test set: Average loss: 2.2730 
Accuracy: 3873/10000 (38.73%)

Round 1073, Train average loss 0.079 Test accuracy 38.730
lr= 0.0016

Test set: Average loss: 2.3732 
Accuracy: 3832/10000 (38.32%)

Round 1074, Train average loss 0.079 Test accuracy 38.320
lr= 0.0016

Test set: Average loss: 2.2826 
Accuracy: 3936/10000 (39.36%)

Round 1075, Train average loss 0.084 Test accuracy 39.360
lr= 0.0016

Test set: Average loss: 2.3320 
Accuracy: 3884/10000 (38.84%)

Round 1076, Train average loss 0.086 Test accuracy 38.840
lr= 0.0016

Test set: Average loss: 2.4294 
Accuracy: 3734/10000 (37.34%)

Round 1077, Train average loss 0.083 Test accuracy 37.340
lr= 0.0016

Test set: Average loss: 2.4472 
Accuracy: 3828/10000 (38.28%)

Round


Test set: Average loss: 2.6360 
Accuracy: 3585/10000 (35.85%)

Round 1133, Train average loss 0.076 Test accuracy 35.850
lr= 0.0016

Test set: Average loss: 2.4432 
Accuracy: 3698/10000 (36.98%)

Round 1134, Train average loss 0.079 Test accuracy 36.980
lr= 0.0016

Test set: Average loss: 2.5882 
Accuracy: 3728/10000 (37.28%)

Round 1135, Train average loss 0.073 Test accuracy 37.280
lr= 0.0016

Test set: Average loss: 2.3580 
Accuracy: 3917/10000 (39.17%)

Round 1136, Train average loss 0.071 Test accuracy 39.170
lr= 0.0016

Test set: Average loss: 2.4074 
Accuracy: 3755/10000 (37.55%)

Round 1137, Train average loss 0.070 Test accuracy 37.550
lr= 0.0016

Test set: Average loss: 2.2923 
Accuracy: 3922/10000 (39.22%)

Round 1138, Train average loss 0.079 Test accuracy 39.220
lr= 0.0016

Test set: Average loss: 2.3632 
Accuracy: 3900/10000 (39.00%)

Round 1139, Train average loss 0.077 Test accuracy 39.000
lr= 0.0016

Test set: Average loss: 2.4530 
Accuracy: 3797/10000 (37.97%)

Round


Test set: Average loss: 2.4143 
Accuracy: 3793/10000 (37.93%)

Round 1195, Train average loss 0.070 Test accuracy 37.930
lr= 0.0016

Test set: Average loss: 2.4894 
Accuracy: 3693/10000 (36.93%)

Round 1196, Train average loss 0.070 Test accuracy 36.930
lr= 0.0016

Test set: Average loss: 2.3253 
Accuracy: 3853/10000 (38.53%)

Round 1197, Train average loss 0.077 Test accuracy 38.530
lr= 0.0016

Test set: Average loss: 2.3487 
Accuracy: 3841/10000 (38.41%)

Round 1198, Train average loss 0.080 Test accuracy 38.410
lr= 0.0016

Test set: Average loss: 2.5827 
Accuracy: 3659/10000 (36.59%)

Round 1199, Train average loss 0.064 Test accuracy 36.590


# 1. Reconstruction

In [6]:
from models.Nets import *
from utils.functions import *

net_glob = CNNCifar(args)
net_glob = net_glob.cuda()

net_glob.train()
# copy weights
w_glob = net_glob.state_dict()

d = tensor_dim(w_glob)

print(d)

62006


In [7]:
p = 0
N = 40
K = 8

args.local_ep=1

N_trials = 1
Max_iter = 100

lr_array = [0.01]


starting_iter_array = [599, 799, 999]

recon_array_proposed = []
recon_array_random = []

gain_array = []

for ii in range(len(starting_iter_array)):
    starting_iter = starting_iter_array[ii]



    net_glob = CNNCifar(args)
    net_glob = net_glob.cuda()

    PATH = "./save_models/CIFAR10_NonIID_CNN_N40_K8_net_glob_iter"+str(starting_iter)
    net_glob.load_state_dict(torch.load(PATH))
    net_glob.eval()

    acc_test, loss_test = test_img(net_glob, dataset_test, args)


    acc_test_arr_v2  = np.zeros((len(lr_array), N_trials, Max_iter))
    loss_test_arr_v2 = np.zeros((len(lr_array), N_trials, Max_iter))



    P_random = []


    for trial_idx in range(N_trials):


        for lr_idx in range(len(lr_array)):

            print()
            print('Learning Rate =',args.lr)
            print()
    #         net_glob = CNNMnist2(args)
    #         net_glob = net_glob.cuda()
    #         print(net_glob)

            net_glob.train()

            # copy weights
            w_glob = net_glob.state_dict()

    #         w_glob_array = []
    #         w_locals_array = []

            w_locals_array_np_v2 = np.zeros((Max_iter,N,d))
            w_glob_array_np_v2 = np.zeros((Max_iter,d))

            w_glob_array = []

            for iter in range(Max_iter): #args.epochs

                args.lr = lr_array[lr_idx]/(starting_iter)
    #             if iter >= 200:
    #                 args.lr = lr_array[lr_idx] * 0.1
    #             elif iter >= 300:
    #                 args.lr = lr_array[lr_idx] * 0.01

                w_locals, loss_locals = [], []
                w_locals_all = []

    # #             u = np.random.binomial(1, 1-p, size=(N))
    #             u = np.ones((N,))
    #             for u_idx in range(N):
    #                 p_sel = p_per_user[u_idx]
    #                 u[u_idx] = np.random.binomial(1, 1-p_sel, size=1)[0]

    #             result = np.where(u == 1)

                ###############################
                # 1. Random Selection
                ###############################
                idxs_users = np.random.choice(N, K, replace=False)

                p_tmp = np.zeros(N)
                p_tmp[idxs_users] = 1

                P_random.append(p_tmp)

    #             print('Learning Rate =',args.lr)
            #     idxs_users = np.random.choice(range(N), K, replace=False)
                for idx in range(N):
            #         print(idx)
                    local = LocalUpdate(args=args, dataset=dataset_train, idxs=dict_users[idx])
                    w, loss = local.train(net=copy.deepcopy(net_glob).to(args.device))

                    w_locals_all.append(copy.deepcopy(w))
                    loss_locals.append(copy.deepcopy(loss))

                    if idx in idxs_users:
                        w_locals.append(copy.deepcopy(w))

                    stt_pos = 0
                    for k in w.keys():
                        tmp1 = w[k].cpu().detach().numpy()
                        cur_shape = tmp1.shape
                        _d = np.prod(cur_shape)

                        end_pos = stt_pos + _d

    #                     w_glob_array_np[iter,stt_pos:end_pos] = np.reshape(tmp1,(_d,))        


                        w_locals_array_np_v2[iter,idx,stt_pos:end_pos] = np.reshape(tmp1,(_d,))

                        stt_pos = end_pos




                # update global weights
                w_glob = FedAvg(w_locals)


                stt_pos = 0
                for k in w_glob.keys():
                    tmp2 = w_glob[k].cpu().detach().numpy()
                    cur_shape = tmp2.shape
                    _d = np.prod(cur_shape)

                    end_pos = stt_pos + _d

    #                 print(_d, stt_pose, end_pos)

                    w_glob_array_np_v2[iter,stt_pos:end_pos] = np.reshape(tmp2,(_d,))

                    stt_pos = end_pos


    #             w_locals_array.append(w_locals_all)
                w_glob_array.append(w_glob)

                ModelDiff_tensor(net_glob.state_dict(), w_glob_array[iter])     

                # copy weight to net_glob
                if iter < 1000:
                    print('net_glob is updated !!')
                    net_glob.load_state_dict(w_glob)
    #             else:
    #                 net_glob.load_state_dict(w_glob_prev)

                # print loss
                loss_avg = sum(loss_locals) / len(loss_locals)

        #         loss_train.append(loss_avg)

                acc_test, loss_test = test_img(net_glob, dataset_test, args)
                acc_test_arr_v2[lr_idx][trial_idx][iter]  = acc_test
                loss_test_arr_v2[lr_idx][trial_idx][iter] = loss_test
                if iter % 1 ==0:
                    print('Round {:3d}, Train average loss {:.3f} Test accuracy {:.3f}'.format(iter, loss_avg,acc_test))
                #print(loss_train)

#                 if iter % 100 == 99:
#                     PATH = "./save_models/MNIST_NonIID_CNN_N40_K8_net_glob_iter"+str(1400+iter)
#                     torch.save(net_glob.state_dict(), PATH)
                    
    grad_locals_array_np_v2 = np.zeros((100,N,d))
    grad_glob_array_np_v2 = np.zeros((100,d))

    for i in range(1, 99):
    #     print(i)
        grad_locals_array_np_v2[i+1,:,:] = (w_locals_array_np_v2[i,:,:] - w_glob_array_np_v2[i-1,:])*1400

        grad_glob_array_np_v2[i-1,:] = (w_glob_array_np_v2[i,:] - w_glob_array_np_v2[i-1,:])*1400

        ModelDiff_np(grad_locals_array_np_v2[i+1,0,:], grad_glob_array_np_v2[i-1,:])
        
    P_random = np.array(P_random)

    print(np.shape(P_random))

    print(np.sum(P_random, axis=0))
    
    # Pseudo Inversion

    offset = 1
    P_random_tmp = P_random[40+offset:90+offset,:]

    PT = P_random_tmp.transpose()

    print(np.shape(PT))

    PTP = np.matmul(P_random_tmp.transpose(), P_random_tmp)

    print(np.linalg.matrix_rank(PTP))

    PTP_inv=np.linalg.pinv(PTP)


    # print(np.shape(PT), np.shape(w_glob_array_np[10:60,:]))
    Pw_glob = np.matmul(PT, grad_glob_array_np_v2[40:90,:])
    grad_recon_np = K * np.matmul(PTP_inv, Pw_glob)

    # ModelDiff_np(w_locals_array_np[-1,1,:], w_recon_np[1])
    l2_diff = np.zeros((N))
    l2_diff_ = np.zeros((N))
    for i in range(N):
        if i == N-1:
            l2_diff_[i] = ModelDiff_np(grad_locals_array_np_v2[40,i,:], (grad_recon_np[i-1]+grad_recon_np[i])/2)
        else:
            l2_diff_[i] = ModelDiff_np(grad_locals_array_np_v2[40,i,:], (grad_recon_np[i]+grad_recon_np[i+1])/2)
    print()
    print()
    for i in range(N):
        l2_diff[i] = ModelDiff_np(grad_locals_array_np_v2[50,i,:], (grad_recon_np[i]))
        
    gain = np.sum(l2_diff_)/np.sum(l2_diff)

    print(np.sum(l2_diff_)/N)
    print(np.sum(l2_diff)/N)

    print(np.sum(l2_diff_)/np.sum(l2_diff))
    
    recon_array_proposed.append(l2_diff_)
    
    recon_array_random.append(l2_diff)
    
    gain_array.append(gain)


Test set: Average loss: 2.2089 
Accuracy: 3644/10000 (36.44%)


Learning Rate = 0.03

[[123.49016998]] [[123.48791882]] [[2.59083795e-06]]
net_glob is updated !!

Test set: Average loss: 2.2040 
Accuracy: 3646/10000 (36.46%)

Round   0, Train average loss 1.742 Test accuracy 36.460
[[123.48791882]] [[123.4855979]] [[4.76466994e-06]]
net_glob is updated !!

Test set: Average loss: 2.1982 
Accuracy: 3654/10000 (36.54%)

Round   1, Train average loss 1.738 Test accuracy 36.540
[[123.4855979]] [[123.48343926]] [[2.97452114e-06]]
net_glob is updated !!

Test set: Average loss: 2.1925 
Accuracy: 3665/10000 (36.65%)

Round   2, Train average loss 1.734 Test accuracy 36.650
[[123.48343926]] [[123.48057445]] [[1.12189689e-05]]
net_glob is updated !!

Test set: Average loss: 2.1825 
Accuracy: 3664/10000 (36.64%)

Round   3, Train average loss 1.729 Test accuracy 36.640
[[123.48057445]] [[123.47805828]] [[5.61597041e-06]]
net_glob is updated !!

Test set: Average loss: 2.1749 
Accuracy: 3677/100


Test set: Average loss: 2.0732 
Accuracy: 3775/10000 (37.75%)

Round  41, Train average loss 1.637 Test accuracy 37.750
[[123.40058355]] [[123.39907821]] [[6.06731831e-06]]
net_glob is updated !!

Test set: Average loss: 2.0707 
Accuracy: 3778/10000 (37.78%)

Round  42, Train average loss 1.635 Test accuracy 37.780
[[123.39907821]] [[123.39697535]] [[4.21568608e-06]]
net_glob is updated !!

Test set: Average loss: 2.0679 
Accuracy: 3789/10000 (37.89%)

Round  43, Train average loss 1.633 Test accuracy 37.890
[[123.39697535]] [[123.39495957]] [[1.59910838e-06]]
net_glob is updated !!

Test set: Average loss: 2.0681 
Accuracy: 3777/10000 (37.77%)

Round  44, Train average loss 1.630 Test accuracy 37.770
[[123.39495957]] [[123.39324806]] [[1.60047193e-06]]
net_glob is updated !!

Test set: Average loss: 2.0677 
Accuracy: 3776/10000 (37.76%)

Round  45, Train average loss 1.631 Test accuracy 37.760
[[123.39324806]] [[123.39162277]] [[8.8009499e-06]]
net_glob is updated !!

Test set: Avera


Test set: Average loss: 2.0261 
Accuracy: 3812/10000 (38.12%)

Round  83, Train average loss 1.599 Test accuracy 38.120
[[123.32025623]] [[123.31841323]] [[2.62544913e-06]]
net_glob is updated !!

Test set: Average loss: 2.0256 
Accuracy: 3815/10000 (38.15%)

Round  84, Train average loss 1.598 Test accuracy 38.150
[[123.31841323]] [[123.3167396]] [[2.26772847e-06]]
net_glob is updated !!

Test set: Average loss: 2.0253 
Accuracy: 3811/10000 (38.11%)

Round  85, Train average loss 1.598 Test accuracy 38.110
[[123.3167396]] [[123.31489963]] [[2.0581365e-06]]
net_glob is updated !!

Test set: Average loss: 2.0244 
Accuracy: 3810/10000 (38.10%)

Round  86, Train average loss 1.598 Test accuracy 38.100
[[123.31489963]] [[123.31335978]] [[1.93350443e-06]]
net_glob is updated !!

Test set: Average loss: 2.0241 
Accuracy: 3810/10000 (38.10%)

Round  87, Train average loss 1.597 Test accuracy 38.100
[[123.31335978]] [[123.31150578]] [[3.0738932e-06]]
net_glob is updated !!

Test set: Average 


Test set: Average loss: 2.4159 
Accuracy: 3686/10000 (36.86%)


Learning Rate = 1.669449081803005e-05

[[129.83220146]] [[129.82993934]] [[5.89974868e-06]]
net_glob is updated !!

Test set: Average loss: 2.4013 
Accuracy: 3694/10000 (36.94%)

Round   0, Train average loss 1.872 Test accuracy 36.940
[[129.82993934]] [[129.82773437]] [[5.23193529e-06]]
net_glob is updated !!

Test set: Average loss: 2.3849 
Accuracy: 3714/10000 (37.14%)

Round   1, Train average loss 1.861 Test accuracy 37.140
[[129.82773437]] [[129.82592509]] [[7.59725855e-06]]
net_glob is updated !!

Test set: Average loss: 2.3727 
Accuracy: 3727/10000 (37.27%)

Round   2, Train average loss 1.848 Test accuracy 37.270
[[129.82592509]] [[129.82358595]] [[7.3566993e-06]]
net_glob is updated !!

Test set: Average loss: 2.3620 
Accuracy: 3732/10000 (37.32%)

Round   3, Train average loss 1.838 Test accuracy 37.320
[[129.82358595]] [[129.82153539]] [[8.61210973e-06]]
net_glob is updated !!

Test set: Average loss: 2.3515 



Test set: Average loss: 2.1939 
Accuracy: 3889/10000 (38.89%)

Round  41, Train average loss 1.701 Test accuracy 38.890
[[129.75286957]] [[129.75127013]] [[2.72248164e-06]]
net_glob is updated !!

Test set: Average loss: 2.1966 
Accuracy: 3888/10000 (38.88%)

Round  42, Train average loss 1.698 Test accuracy 38.880
[[129.75127013]] [[129.7492151]] [[2.95300097e-06]]
net_glob is updated !!

Test set: Average loss: 2.1916 
Accuracy: 3883/10000 (38.83%)

Round  43, Train average loss 1.700 Test accuracy 38.830
[[129.7492151]] [[129.74765639]] [[3.75533712e-06]]
net_glob is updated !!

Test set: Average loss: 2.1931 
Accuracy: 3882/10000 (38.82%)

Round  44, Train average loss 1.696 Test accuracy 38.820
[[129.74765639]] [[129.74565896]] [[2.97887917e-06]]
net_glob is updated !!

Test set: Average loss: 2.1893 
Accuracy: 3875/10000 (38.75%)

Round  45, Train average loss 1.697 Test accuracy 38.750
[[129.74565896]] [[129.74350996]] [[4.22221481e-06]]
net_glob is updated !!

Test set: Averag


Test set: Average loss: 2.1448 
Accuracy: 3882/10000 (38.82%)

Round  83, Train average loss 1.662 Test accuracy 38.820
[[129.68114035]] [[129.67967639]] [[4.91657452e-06]]
net_glob is updated !!

Test set: Average loss: 2.1449 
Accuracy: 3885/10000 (38.85%)

Round  84, Train average loss 1.662 Test accuracy 38.850
[[129.67967639]] [[129.67817691]] [[4.93422076e-06]]
net_glob is updated !!

Test set: Average loss: 2.1428 
Accuracy: 3889/10000 (38.89%)

Round  85, Train average loss 1.663 Test accuracy 38.890
[[129.67817691]] [[129.6765437]] [[1.77764345e-06]]
net_glob is updated !!

Test set: Average loss: 2.1424 
Accuracy: 3883/10000 (38.83%)

Round  86, Train average loss 1.661 Test accuracy 38.830
[[129.6765437]] [[129.67537742]] [[1.86381231e-06]]
net_glob is updated !!

Test set: Average loss: 2.1421 
Accuracy: 3883/10000 (38.83%)

Round  87, Train average loss 1.661 Test accuracy 38.830
[[129.67537742]] [[129.67357572]] [[1.75560227e-06]]
net_glob is updated !!

Test set: Averag


Test set: Average loss: 2.3315 
Accuracy: 3859/10000 (38.59%)


Learning Rate = 1.2515644555694618e-05

[[130.92893587]] [[130.92731682]] [[1.91271051e-06]]
net_glob is updated !!

Test set: Average loss: 2.3252 
Accuracy: 3864/10000 (38.64%)

Round   0, Train average loss 1.731 Test accuracy 38.640
[[130.92731682]] [[130.92604227]] [[3.61671436e-06]]
net_glob is updated !!

Test set: Average loss: 2.3237 
Accuracy: 3862/10000 (38.62%)

Round   1, Train average loss 1.727 Test accuracy 38.620
[[130.92604227]] [[130.92460337]] [[3.86000344e-06]]
net_glob is updated !!

Test set: Average loss: 2.3200 
Accuracy: 3876/10000 (38.76%)

Round   2, Train average loss 1.726 Test accuracy 38.760
[[130.92460337]] [[130.92302161]] [[3.46662004e-06]]
net_glob is updated !!

Test set: Average loss: 2.3160 
Accuracy: 3878/10000 (38.78%)

Round   3, Train average loss 1.723 Test accuracy 38.780
[[130.92302161]] [[130.92154535]] [[2.88511971e-06]]
net_glob is updated !!

Test set: Average loss: 2.3099


Test set: Average loss: 2.1904 
Accuracy: 3948/10000 (39.48%)

Round  41, Train average loss 1.625 Test accuracy 39.480
[[130.8674994]] [[130.86619385]] [[3.93841678e-06]]
net_glob is updated !!

Test set: Average loss: 2.1898 
Accuracy: 3962/10000 (39.62%)

Round  42, Train average loss 1.627 Test accuracy 39.620
[[130.86619385]] [[130.86505152]] [[2.35067844e-06]]
net_glob is updated !!

Test set: Average loss: 2.1886 
Accuracy: 3950/10000 (39.50%)

Round  43, Train average loss 1.626 Test accuracy 39.500
[[130.86505152]] [[130.86353886]] [[5.40802333e-06]]
net_glob is updated !!

Test set: Average loss: 2.1854 
Accuracy: 3949/10000 (39.49%)

Round  44, Train average loss 1.626 Test accuracy 39.490
[[130.86353886]] [[130.86176695]] [[8.04249562e-06]]
net_glob is updated !!

Test set: Average loss: 2.1825 
Accuracy: 3949/10000 (39.49%)

Round  45, Train average loss 1.624 Test accuracy 39.490
[[130.86176695]] [[130.86032812]] [[4.3109453e-06]]
net_glob is updated !!

Test set: Averag


Test set: Average loss: 2.1674 
Accuracy: 3945/10000 (39.45%)

Round  83, Train average loss 1.614 Test accuracy 39.450
[[130.81204253]] [[130.81090359]] [[7.47117701e-06]]
net_glob is updated !!

Test set: Average loss: 2.1687 
Accuracy: 3943/10000 (39.43%)

Round  84, Train average loss 1.613 Test accuracy 39.430
[[130.81090359]] [[130.80947871]] [[3.32539957e-06]]
net_glob is updated !!

Test set: Average loss: 2.1713 
Accuracy: 3940/10000 (39.40%)

Round  85, Train average loss 1.614 Test accuracy 39.400
[[130.80947871]] [[130.80825191]] [[2.08430046e-06]]
net_glob is updated !!

Test set: Average loss: 2.1707 
Accuracy: 3948/10000 (39.48%)

Round  86, Train average loss 1.616 Test accuracy 39.480
[[130.80825191]] [[130.80694388]] [[4.70651058e-06]]
net_glob is updated !!

Test set: Average loss: 2.1689 
Accuracy: 3949/10000 (39.49%)

Round  87, Train average loss 1.616 Test accuracy 39.490
[[130.80694388]] [[130.80549743]] [[4.76495452e-06]]
net_glob is updated !!

Test set: Aver

[[89.67429957]] [[63.49149079]] [[3.85065238]]
[[93.66718018]] [[92.5743847]] [[1.0236677]]
[[55.61978852]] [[51.44982584]] [[3.15783009]]
[[84.15703747]] [[86.84331962]] [[0.98513798]]
[[60.7931714]] [[65.48420448]] [[0.62778028]]
[[73.99048513]] [[107.23125445]] [[9.3166131]]
[[60.54181121]] [[56.87797312]] [[4.77183194]]
[[94.76899819]] [[123.39512748]] [[5.75341667]]
[[72.57501318]] [[55.7106807]] [[9.38539037]]
[[53.63415169]] [[59.62727488]] [[4.49627261]]
[[95.02044159]] [[93.51545821]] [[5.04852903]]
[[73.23501852]] [[101.81389258]] [[4.91627734]]
[[83.27428558]] [[71.39224184]] [[8.16270621]]
[[71.42505971]] [[50.81032842]] [[7.12186231]]
[[64.80558614]] [[49.56988249]] [[9.70929959]]
[[93.11238685]] [[110.02897648]] [[6.43233996]]
[[71.8345607]] [[61.32232354]] [[3.82208978]]
[[83.07292912]] [[87.3370908]] [[2.38346929]]
[[72.01761681]] [[87.66717766]] [[2.70514916]]
[[66.57393884]] [[51.88205182]] [[1.90956443]]
[[65.39260443]] [[53.15421916]] [[5.42902659]]
[[74.08092182]] 